#### Sequence to Sequence

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

seq_data = [['word', '단어'], ['wood', '나무'],
            ['game', '놀이'], ['girl', '소녀'],
            ['kiss', '키스'], ['love', '사랑']]

def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []
    
    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]
        
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
    
    return input_batch, output_batch, target_batch


In [3]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 100

n_class = n_input = dic_len

In [4]:
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell,
                                             output_keep_prob = 0.5)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                            dtype = tf.float32)

with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, 
                                             output_keep_prob = 0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, 
                                            initial_state = enc_states, dtype = tf.float32)


In [5]:
model = tf.layers.dense(outputs, n_class, activation=None)

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels = targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [7]:
input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})
    print('Epoch: ', '%04d' % (epoch + 1),
          'cost = ', '{:.6f}'.format(loss))

print('최적화 완료!')

Epoch:  0001 cost =  3.709091
Epoch:  0002 cost =  2.589462
Epoch:  0003 cost =  1.645779
Epoch:  0004 cost =  0.905564
Epoch:  0005 cost =  0.625466
Epoch:  0006 cost =  0.374862
Epoch:  0007 cost =  0.415978
Epoch:  0008 cost =  0.159362
Epoch:  0009 cost =  0.215484
Epoch:  0010 cost =  0.140496
Epoch:  0011 cost =  0.180748
Epoch:  0012 cost =  0.129443
Epoch:  0013 cost =  0.189756
Epoch:  0014 cost =  0.202588
Epoch:  0015 cost =  0.109811
Epoch:  0016 cost =  0.060038
Epoch:  0017 cost =  0.087649
Epoch:  0018 cost =  0.041391
Epoch:  0019 cost =  0.043085
Epoch:  0020 cost =  0.051606
Epoch:  0021 cost =  0.060835
Epoch:  0022 cost =  0.013817
Epoch:  0023 cost =  0.074167
Epoch:  0024 cost =  0.021245
Epoch:  0025 cost =  0.088102
Epoch:  0026 cost =  0.011744
Epoch:  0027 cost =  0.008758
Epoch:  0028 cost =  0.045453
Epoch:  0029 cost =  0.025833
Epoch:  0030 cost =  0.002582
Epoch:  0031 cost =  0.006670
Epoch:  0032 cost =  0.005177
Epoch:  0033 cost =  0.005703
Epoch:  00

In [8]:
def translate(word):
    seq_data = [word, 'P' * len(word)]
    
    input_batch, output_batch, target_batch = make_batch([seq_data])
    
    prediction = tf.argmax(model, 2)
    
    result = sess.run(prediction, feed_dict = {enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    
    decoded = [char_arr[i] for i in result[0]]
    
    end = decoded.index('E')
    translated = ''.join(decoded[:end])
    
    return translated

In [9]:
print('\n === 번역 테스트 ===')

print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))
print('kiss ->', translate('kiss'))



 === 번역 테스트 ===
word -> 단어
wodr -> 단무
love -> 사랑
loev -> 사랑
abcd -> 사랑
kiss -> 키스
